In [1]:
from pyspark.sql import SparkSession
import requests
import json
from datetime import datetime

In [2]:
# Initialize Spark session
spark = SparkSession.builder.appName("Time_services").master("spark://spark-master:7077").config("spark.cores.max", "2").config("spark.executor.memory", "512m").config("spark.eventLog.enabled", "true").config("spark.eventLog.dir", "file:///opt/workspace/events").getOrCreate()

# Read JSON files from HDFS
df = spark.read.json("all_papers.json")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/06/22 09:56:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


24/06/22 09:56:42 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
def transform_row(row):
    json_id = row['id']
    date = row['publishedDate']
    date_object = datetime.strptime(date, "%Y-%m-%dT%H:%M:%S")
    # Extract day of week
    day_of_week = date_object.strftime("%A")
    month = date_object.strftime("%B")

    return (json_id, month, day_of_week)

In [4]:
transformed_data = df.rdd.map(transform_row)

transformed_df = transformed_data.toDF(["id", "month", "day_of_week"])    

In [5]:
# Save DataFrame to CSV
transformed_df.coalesce(1).write.csv("tabular/paper_dates", header=True, mode="overwrite")

In [6]:
spark.stop()